In [ ]:
# Import basic Python modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True)

#Read in csv of cleaned data for the full regression(71 predictors: df1)
df = pd.read_csv('Model1.csv')
print(df.shape)
df.head()


## Set up variables and scale the features

In [ ]:
# Import sklearn regression modules
from sklearn.linear_model import LinearRegression

#Set up variables: X, y
df1= df.drop('drugmort', axis = 1)
X = df1.values
y = df['drugmort'].values


In [ ]:
# Import scale
from sklearn.preprocessing import scale

# Scale the features
X = scale(X)
y = scale(y)


## Linear regression - full, untransformed, scaled: Model 1 (from R)

In [ ]:
# Import necessary modules
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

# Create the regressor: reg_all
reg_all = LinearRegression()

# Fit the regressor to the training data
reg_all.fit(X_train, y_train)

# Predict on the test data: y_pred
y_pred = reg_all.predict(X_test)

# Compute and print R^2 and RMSE
print("R^2 training set: {}".format(reg_all.score(X_train, y_train)))

rmsemod1 = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error testing set: {}".format(rmsemod1))

## 10-Fold cross validation on full, untransformed linear model

In [ ]:
# Import the necessary modules
from sklearn.model_selection import cross_val_score

# Compute 10-fold cross-validation scores: cv_reg
cv_reg = cross_val_score(reg_all, X, y, cv=10)

# Print the 10-fold cross-validation scores
print(cv_reg)

print("Average 10-Fold CV Score: {}".format(np.mean(cv_reg)))

## Ridge Regression: Full/untransformed data

### 1) Determine the best alpha level

In [ ]:
def display_plot(cv_scores, cv_scores_std):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(alpha_space, cv_scores)

    std_error = cv_scores_std / np.sqrt(10)

    ax.fill_between(alpha_space, cv_scores + std_error, cv_scores - std_error, alpha=0.2)
    ax.set_ylabel('CV Score +/- Std Error')
    ax.set_xlabel('Alpha')
    ax.axhline(np.max(cv_scores), linestyle='--', color='.5')
    ax.set_xlim([alpha_space[0], alpha_space[-1]])
    ax.set_xscale('log')
    plt.show()

In [ ]:
# Import necessary modules
from sklearn.linear_model import Ridge

# Setup the array of alphas and lists to store scores
alpha_space = np.logspace(-4, 0, 50)
ridge_scores = []
ridge_scores_std = []

# Create a ridge regressor: ridge
ridge = Ridge(normalize=True)

# Compute scores over range of alphas
for alpha in alpha_space:

    # Specify the alpha value to use: ridge.alpha
    ridge.alpha = alpha
    
    # Perform 10-fold CV: ridge_cv_scores
    ridge_cv_scores = cross_val_score(ridge, X, y, cv=10)
    
    # Append the mean of ridge_cv_scores to ridge_scores
    ridge_scores.append(np.mean(ridge_cv_scores))
    
    # Append the std of ridge_cv_scores to ridge_scores_std
    ridge_scores_std.append(np.std(ridge_cv_scores))

# Display the plot
display_plot(ridge_scores, ridge_scores_std)

### 2) Run Ridge regression on full/untransformed

In [ ]:
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

# Create the regressor: ridge
ridge1 = Ridge(alpha=0.08, normalize=True)

# Fit the regressor to the training data
ridge1.fit(X_train, y_train)

# Predict on the test data: y_pred
y_pred = ridge1.predict(X_test)

# Compute and print R^2 and RMSE
print("R^2 training set: {}".format(ridge1.score(X_train, y_train)))
rmse_ridge1 = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error testing set: {}".format(rmse_ridge1))

### 3) 10-fold cross-validation:  Ridge on full/untransformed data

In [ ]:
# Compute 10-fold cross-validation scores: cv_ridge1
cv_ridge1 = cross_val_score(ridge1, X, y, cv=10)

# Print the 10-fold cross-validation scores
print(cv_ridge1)

print("Average 10-Fold CV Score: {}".format(np.mean(cv_ridge1)))

## Linear and Ridge regressions using transformed data (log)

In [ ]:
#Read in csv of transformed data: dftrans
dftrans = pd.read_csv('Model3.csv')
print(dftrans.shape)
dftrans.head()


In [ ]:
#This is the df for the linear regression: Step, Transformed

df35 = dftrans[['drugmort','aamort', 'chmort' , 'infmort' , 'mentdistr' , 
    'foodins' , 'mvmort' , 'uninsure' , 'chuninsure' , 'disconyouth' , 
    'homicide' , 'under18' , 'over65' , 'aframer' , 'amerindian' , 'asian' , 
    'hawaiin' , 'hispanic' , 'rural' , 'lifelost' , 'fairhealth' , 'lowbirth' , 
    'physinactive' , 'excdrinking' , 'alcoholdrive' , 'chlamydia' , 'unemployed' , 
    'chpovertyw' , 'X80income' , 'singparent' , 'socialassoc' , 'severehous' , 
    'commute' , 'new_pcp_ratio' , 'new_mhp_ratio']]
print(df35.shape)
df35.head()

## Set up variables and scale the features: Full/transformed, step/transformed

In [ ]:
#Set up variables for Full/transformed and Step/transformed: Xft, yft, Xst, yst
dftrans2 = dftrans.drop('drugmort', axis = 1)
Xft = dftrans2.values
yft = dftrans['drugmort'].values

df35_2 = df35.drop('drugmort', axis = 1)
Xst = df35_2.values
yst = df35['drugmort'].values


In [ ]:
# Scale the features
Xft = scale(Xft)
yft = scale(yft)

Xst = scale(Xst)
yst = scale(yst)

## Linear regression - Step, transformed, scaled

In [ ]:
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(Xst, yst, test_size = 0.3, random_state=42)

# Create the regressor: reg_step
reg_step = LinearRegression()

# Fit the regressor to the training data
reg_step.fit(X_train, y_train)

# Predict on the test data: y_pred
y_pred = reg_step.predict(X_test)

# Compute and print R^2 and RMSE
print("R^2 training set: {}".format(reg_step.score(X_train, y_train)))

rmse_step = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error testing set: {}".format(rmse_step))


In [ ]:
# Compute 10-fold cross-validation scores: cv_step
cv_step = cross_val_score(reg_step, Xst, yst, cv=10)

# Print the 10-fold cross-validation scores
print(cv_step)

print("Average 10-Fold CV Score: {}".format(np.mean(cv_step)))

## Ridge Regression: Full/transformed data

### 1) Determine the best alpha level

In [ ]:
# Setup the array of alphas and lists to store scores
alpha_space = np.logspace(-4, 0, 50)
ridge_scores = []
ridge_scores_std = []

# Create a ridge regressor: ridge
ridge = Ridge(normalize=True)

# Compute scores over range of alphas
for alpha in alpha_space:

    # Specify the alpha value to use: ridge.alpha
    ridge.alpha = alpha
    
    # Perform 10-fold CV: ridge_cv_scores
    ridge_cv_scores = cross_val_score(ridge, Xft, yft, cv=10)
    
    # Append the mean of ridge_cv_scores to ridge_scores
    ridge_scores.append(np.mean(ridge_cv_scores))
    
    # Append the std of ridge_cv_scores to ridge_scores_std
    ridge_scores_std.append(np.std(ridge_cv_scores))

# Display the plot
display_plot(ridge_scores, ridge_scores_std)

Use 0.08 as before

### 2) Run Ridge regression on full/transformed data

In [ ]:

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(Xft, yft, test_size = 0.3, random_state=42)

# Create the regressor: ridge
ridge2 = Ridge(alpha=0.08, normalize=True)

# Fit the regressor to the training data
ridge2.fit(X_train, y_train)

# Predict on the test data: y_pred
y_pred = ridge2.predict(X_test)

# Compute and print R^2 and RMSE
print("R^2 training set: {}".format(ridge2.score(X_train, y_train)))
rmse_ridge2 = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error testing set: {}".format(rmse_ridge2))

### 3) 10-fold cross-validation:  Ridge on full/transformed data

In [ ]:
# Compute 10-fold cross-validation scores: cv_ridge2
cv_ridge2 = cross_val_score(ridge2, Xft, yft, cv=10)

# Print the 10-fold cross-validation scores
print(cv_ridge2)

print("Average 10-Fold CV Score: {}".format(np.mean(cv_ridge2)))

## CONCLUSIONS:  

The best R^2 score was with the Ridge regression model with the full, untransformed data (0.57417113). The best RMSE score and the best CV-score were obtained with the Linear regression model with the step-transformed data (0.649084, 0.4530938). Because the purpose of this study is to predict missing values, the Linear regression model with the step-transformed data is the best choice. This model will be used to predict the missing values and produce a completed, estimated map.

## Train a final model

Run the Linear regresssion model above on the complete step-transformed dataframe: df35

In [ ]:
model = LinearRegression()
model_step = model.fit(Xst, yst)

print("R^2 final model: {}".format(model_step.score(Xst, yst)))

## Predict missing drug overdose mortality values

In [ ]:
#Read in csv of cleaned data for testing
df_test = pd.read_csv('VLynn_DrugOverdose_Test.csv')
df35_test = pd.DataFrame(data = df_test, columns = ['aamort', 'chmort', 'infmort', 'mentdistr', 'foodins', 'mvmort',
       'uninsure', 'chuninsure', 'disconyouth', 'homicide', 'under18',
       'over65', 'aframer', 'amerindian', 'asian', 'hawaiin', 'hispanic',
       'rural', 'lifelost', 'fairhealth', 'lowbirth', 'physinactive',
       'excdrinking', 'alcoholdrive', 'chlamydia', 'unemployed', 'chpovertyw',
       '80income', 'singparent', 'socialassoc', 'severehous', 'commute',
       'new_pcp_ratio', 'new_mhp_ratio'])
df_test.shape

In [ ]:
#Take the log for all the log-normal variables.
df35_logtest = pd.DataFrame(data = df35_test, columns = ['aamort', 'chmort', 'infmort', 'mentdistr', 'mvmort', 'homicide',
       'over65', 'lifelost', 'fairhealth', 'lowbirth', 'chlamydia',
       'singparent', 'severehous', 'new_mhp_ratio'])

df35_logtest = np.log(df35_logtest)
df35_logtest.shape


In [ ]:
#Create dataframe of numerical variables that are not log-normal
df35_var = pd.DataFrame(data = df35_test, columns =['foodins', 'uninsure', 'chuninsure', 'disconyouth', 'under18',
       'aframer', 'amerindian', 'asian', 'hawaiin', 'hispanic', 'rural',
       'physinactive', 'excdrinking', 'alcoholdrive', 'unemployed',
       'chpovertyw', '80income','socialassoc', 'commute', 'new_pcp_ratio'])

#Perform inner join with log-variables
left = df35_logtest
right = df35_var

df_predict = pd.merge(left, right, left_index=True, right_index=True)

df_predict[['foodins', 'physinactive','excdrinking', 'commute']] = df_predict[['foodins', 'physinactive','excdrinking', 'commute']].astype(float)


In [ ]:

#Check for infinite values
print(df_predict.columns.to_series()[np.isinf(df_predict).any()])
print(df_predict.index[np.isinf(df_predict).any(1)])

#Replace infinite value with NaN, then fillna with column mean
pd.options.mode.use_inf_as_na = True
df_predict['singparent'].fillna(df_predict['singparent'].mean(),inplace=True)

#Set up variables from testing dataframe: 
Xpred = df_predict.values

#Scale variables
Xpred = scale(Xpred)

# Predict on the test data: y_pred
Ypred = model_step.predict(Xpred)
print(type(Ypred))
print(len(Ypred))
print(Ypred[:15])


In [ ]:
#Unscale the predicted values
mean_of_array = Ypred.mean(axis=0)
std_of_array = Ypred.std(axis=0)

Yunscaled = (Ypred * std_of_array) + mean_of_array

#Unlog the predicted unscaled values
predictedY = np.exp(Yunscaled)
print(Ypred[:15])
print('')
print (Yunscaled[:15])
print('')
print(predictedY[:15])



In [ ]:
#Convert the series to a dataframe and get summary stats for predicted values
df_predY = pd.DataFrame(data = predictedY, columns=['drugmort'])
df_predY['drugmort'] = df_predY['drugmort'].round(0)
df_predY['drugmort'].describe()


In [ ]:
#compare to summary stats for training data values
df_train = pd.read_csv('VLynn_DrugOverdose_Train.csv')
df_train.head()
df_train['drugmort'].describe()

These look different so I will compare graphically.

In [ ]:
import scipy
from scipy import stats

plt.figure(figsize=(8,4))
plt.title('Density Plots for Nonpredicted and Predicted Variables',size = 20, y=1.02)
sns.kdeplot(df_train['drugmort'], label="drugmort unpredicted")
sns.kdeplot(df_predY['drugmort'], label="drugmort unpredicted")
plt.xlabel("Drug mortality")

plt.legend();
plt.show()

In [ ]:
#Join training and testing sets
df_test['drugmort'] = df_predY['drugmort']
df_joined = pd.concat([df_train, df_test], ignore_index=True)

df_joined.shape

The shape of the distribution of predicted values is similar, right skewed with most of the data near the lower end of the values. However, the model made predictions with more values at the extremes. The standard deviations are similar but the predicted values have a lower mean and a much higher maximum values. There are more outliers in the predicted values. This means that most likely the predicted values will be either lower or higher than actual values. 

There are several things that could be done to improve this model. The first would be to include interaction terms, which were not tested in this study. The second thing would be to consider generalized linear regression models that may better handle this data, as it included variables that were not normally distributed. The last would be to include the Elastic Net in the group of tested models. 

## Join predicted data to original map data to create predicted map

In [ ]:
#Read csv with original mapdata: Mapdata.csv
df_map = pd.read_csv('Mapdata.csv')
df_map.rename(columns={"State": "state", "County": "county"}, inplace=True)

#Sort both by state and county
df_joined.sort_values(['state', 'county'],inplace=True)
df_map.sort_values(['state', 'county'],inplace=True)

result = pd.merge(df_map, df_joined, how='inner', on=['state', 'county'])
result.to_csv('VLynn_DrugOverdose_Finalmap.csv')
